# Shark alley bathymetric survey 20/11/2024

Make a 3D plot from single beam echo sounder data!

### Import neccessary libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy.signal import medfilt

#### Read and check the data:

In [ ]:
data = pd.read_csv('data/241119_shark_alley_survey.csv')
data.head() # This allows us to look at the top of the dataframe. 

In [ ]:
plt.plot(data['Depth'])

#### The data is messy!

There are some zero values and some very deep measurements that seem unrealistic. 

The code below will clean the data.

In [ ]:
# Step 1: Remove zeros
data = data[data['Depth'] <= -1].copy()

# Step 2: Apply median filter to remove spikes
window_size = 7  # Choose an odd number for the window size (e.g., 3, 5, 7)

# Step 3: Apply rolling average to smooth the data further
rolling_window = 5  # Choose the rolling window size

data['depth_despiked'] = medfilt(data['Depth'], kernel_size=window_size)
data['depth_smoothed'] = data['depth_despiked'].rolling(window=rolling_window, center=True).mean()
data['depth_smoothed'].plot()



Much better! 

In [ ]:

plt.figure(figsize=(8, 6))
scatter = plt.scatter(data['East'], data['North'], c=data['depth_smoothed'], cmap='viridis', s=10)
plt.colorbar(scatter, label='Depth')
plt.xlabel('East')
plt.ylabel('North')
plt.title('Shark Alley Survey')
plt.grid(True)

plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

# --- Input: merged_data with columns 'East', 'North', 'Depth applied elevation'
df = data[['East', 'North', 'depth_smoothed']].dropna().copy()

# If duplicate xy points exist, average their z to avoid issues
df = df.groupby(['East', 'North'], as_index=False)['depth_smoothed'].mean()

# Coords and values
xy = df[['East', 'North']].to_numpy()
z  = df['depth_smoothed'].to_numpy()

# --- Make a grid (adjust nx, ny for resolution)
nx, ny = 250, 250
xmin, xmax = xy[:,0].min(), xy[:,0].max()
ymin, ymax = xy[:,1].min(), xy[:,1].max()

gx = np.linspace(xmin, xmax, nx)
gy = np.linspace(ymin, ymax, ny)
GX, GY = np.meshgrid(gx, gy)

# --- Interpolate (method='linear' | 'cubic' | 'nearest')
Z = griddata(points=xy, values=z, xi=(GX, GY), method='linear')

# --- Plot
plt.figure(figsize=(7.5, 7))
im = plt.pcolormesh(GX, GY, Z, shading='auto', vmin=-7, vmax=-1)
plt.scatter(xy[:,0], xy[:,1], s=8, c='k', alpha=0.6, label='Points')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel('East'); plt.ylabel('North')
cbar = plt.colorbar(im); cbar.set_label('Depth applied elevation')
plt.title('Interpolated surface (griddata: linear)')
plt.legend(frameon=True)
plt.tight_layout()
plt.show()

# --- Plot
plt.figure(figsize=(7.5, 7))
im = plt.pcolormesh(GX, GY, Z, shading='auto', vmin=-7, vmax=-1)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel('East'); plt.ylabel('North')
cbar = plt.colorbar(im); cbar.set_label('Depth applied elevation')
plt.title('Interpolated surface (griddata: linear)')
plt.legend(frameon=True)
plt.tight_layout()
plt.show()


In [ ]:
import contextily as ctx

# --- Coordinate reference system (CRS) ---------------------------------------
# Purpose: tell contextily what projection your East/North coordinates use.
# Replace with the correct EPSG for your data (example below is UTM 56S).
crs_code = "EPSG:32756"

# --- Create figure & plot your gridded surface -------------------------------
# Purpose: draw the interpolated bathymetry as a coloured mesh.
fig, ax = plt.subplots(figsize=(7.5, 7))
im = ax.pcolormesh(
    GX, GY, Z,
    shading='auto',
    vmin=-7, vmax=-0.5,
    rasterized=True,         # lighter PDFs while keeping axes/text vector
    cmap='viridis'           # choose your palette; 'viridis' is clear for depth
)
ax.set_aspect('equal', adjustable='box')
ax.set_xlabel('East'); ax.set_ylabel('North')
ax.set_title('Interpolated surface (griddata: linear)')

# --- Colourbar ---------------------------------------------------------------
# Purpose: give readers the value scale for the mesh colours.
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Depth applied elevation')

# --- Compute data bounds & apply a small zoom-out ----------------------------
# Purpose: expand the view a touch so the basemap context shows around the edges.
x0, x1 = np.nanmin(GX), np.nanmax(GX)
y0, y1 = np.nanmin(GY), np.nanmax(GY)
pad = 0.50  # 5% extra space; try 0.02–0.10 as needed
dx = (x1 - x0) * pad
dy = (y1 - y0) * pad
ax.set_xlim(x0 - dx, x1 + dx)
ax.set_ylim(y0 - dy, y1 + dy)

# --- Add basemap underlay ----------------------------------------------------
# Purpose: place simple satellite imagery beneath your data for context.
# Tip: if the imagery competes with colours, lower alpha or switch provider.
ctx.add_basemap(
    ax,
    crs=crs_code,
    source=ctx.providers.Esri.WorldImagery,  # swap for OpenStreetMap if preferred
    alpha=0.9
)

# Some contextily versions adjust limits; re-assert if needed:
ax.set_xlim(x0 - dx, x1 + dx)
ax.set_ylim(y0 - dy, y1 + dy)

# --- Final tidy-ups ----------------------------------------------------------
# Purpose: remove padding and render.
ax.margins(0)
plt.tight_layout()
plt.show()